## Import 

In [ ]:
cd ..

In [ ]:
pwd

In [ ]:
import os, sys, pickle

import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
import plotly.express as px

from sklearn.model_selection import * 
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.impute import * 
from sklearn.preprocessing import * 
from sklearn.ensemble import * 
# from xgboost import XGBClassifier

import shap

## Load data

In [ ]:
train = "./data/cleaned/df_train.csv"
df_train = pd.read_csv(train)
df_train.head()

In [ ]:
test = "./data/cleaned/df_test.csv"
df_test = pd.read_csv (test)

In [ ]:
## Model Load

In [ ]:
with open("./models/model.pk", "rb") as f : 
    model = pickle.load(f)

model

In [ ]:
with open("./models/preprocessor.pk", "rb") as f : 
    preprocessor = pickle.load(f)

preprocessor

In [ ]:
with open("./models/imputer.pk", "rb") as f : 
    imputer = pickle.load(f)

imputer

In [ ]:
with open("./models/estimator.pk", "rb") as f : 
    estimator = pickle.load(f)

estimator

In [ ]:
model

# Test model

In [ ]:
df_train

In [ ]:
X_train = df_train.drop(columns="Survived")
X_train

In [ ]:
X_train_PassengerId = df_train.PassengerId.values
y_train = df_train.Survived.values
X_train = df_train.drop(columns=["Survived",], inplace=False)
X_train.head()

In [ ]:
model.predict(X_train)[:10]

In [ ]:
model.score(X_train, y_train)

# EXPLAIN

In [ ]:
feat = pd.Series(model.feature_importances_, index=X_train.columns)# .sort_values(ascending=False)
px.bar(feat)

In [ ]:
feat = pd.Series(estimator.feature_importances_, index=preprocessor.get_feature_names_out() )# .sort_values(ascending=False)
px.bar(feat)

In [ ]:

# feat = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
# px.bar(feat)

In [ ]:
explain = pd.Series(estimator.feature_importances_, index= estimator.feature_names_in_)
explain

In [ ]:
explain = {k: v.round(2) for v, k in zip(model.feature_importances_, model.feature_names_in_)}
explain


In [ ]:
explainer = shap.TreeExplainer(estimator) # Explainer
explainer

In [ ]:
with open("./models/shap.pk", "wb") as f : 
    pk = pickle.dumps(explainer)
    f.write(pk)

In [ ]:
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar",show=True)

In [ ]:
fig = shap.summary_plot(shap_values, X_train, show=False)
plt.savefig('./assets/shap.png')

In [ ]:
shap.initjs()


In [ ]:
X_train.iloc[0]

In [ ]:
df_train.iloc[0]

In [ ]:
def extract_vect(
    _id: int,
    df: pd.DataFrame,
    id_col: str = "PassengerId",
    target_col: str = "Survived",
) -> tuple:
    """extract from a df"""

    _id = int(_id)

    X = df.copy().loc[df[id_col] == _id]
    assert len(X) == 1

    id_val = X[id_col].iloc[0]
    target_val = X[target_col].iloc[0]

    drop_cols = [id_col, target_col]
    X = X.drop(columns=drop_cols)
    ser = X.iloc[0].to_dict()

    return ser, id_val, target_val

In [ ]:
_id = df_train.PassengerId.sample(1).iloc[0]
_id

In [ ]:
__X, __Passenger_id, __Survived = extract_vect(_id, df_train)

In [ ]:
pd.Series(__X)

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][0,:], pd.Series(__X), show=False)

In [ ]:
shap_false = shap_values[0]
shap_true = shap_values[1]

In [ ]:
pd.DataFrame(shap_true, columns=X_train. columns).iloc[0].sort_values(ascending=False).head(5).round(2)

In [ ]:
pd.DataFrame(shap_false, columns=X_train. columns).iloc[0].sort_values(ascending=False).head(5).round(2)

In [ ]:
!pwd

In [ ]:
plt.savefig('./assets/shap.png')

In [ ]:
explainer.expected_value[1]

In [ ]:
shap_values = explainer.shap_values(pd.DataFrame([__X,]))

In [ ]:
shap_false = shap_values[0]
shap_true = shap_values[1]

In [ ]:
shap_true = pd.DataFrame(shap_true, columns=X_train. columns).iloc[0].sort_values(ascending=False).head(5).round(2).to_dict()

In [ ]:
shap_false = pd.DataFrame(shap_false, columns=X_train. columns).iloc[0].sort_values(ascending=False).head(5).round(2).to_dict()

In [ ]:
pd.DataFrame(shap_values[0])

In [ ]:
pd.DataFrame(shap_values[1])

In [ ]:
pd.DataFrame(shap_values[2])

In [ ]:
"""
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.savefig('grafic.png')
"""

In [ ]:
"""
shap.plots.waterfall(shap_values[0])
"""

In [ ]:
"""

explainer = shap.TreeExplainer(model, X_train)
shap_values = explainer(X_train)
shap.plots.waterfall(shap_values[0])
"""

In [ ]:
"""
shap.plots.waterfall(shap_values[0])
"""